# Using Neural Networks to Recognize Handwritten Digits

#### We will build a Multi-layer Percepteron as the architecture for our neural network.

#### We will use the MNIST Dataset for training our network. It contains 60,000 training images, and 10,000 test images.

The loading and handling of the training images is done in a seperate file - mnist_loader.py

## Neural Network Version 1

#### We will build our first neural network and use stochastic gradient descent as the learning method. 

##### We will use the sigmoid activation function. The hyperparameters are the no. of epochs, η, no. of neurons/units in each layer and the no. of layers itself.

Before we go on, some notational background:
- Epochs: Stochastic gradient descent works by picking out a randomly chosen mini-batch of training imputs and training the 
  weights and biases with those. Then we pick out another randomly chosen mini-batch and train with those and so on               until we have exhausted the training inputs, which is said to complete a epoch of training
- η/eta - This is the learning rate. If it is too small, training will be very slow and if it is too large, the flactuations             will be wild and convergence to the minimum will be very erratic, if any. 
- No. of layers - We can control the complexity by changing the no. of layers in our network.
- No. of neurons - We will always use 10 neurons in the output layer: one to represent each digit.
- Cost function - The loss function we'll use is the Mean Square Losee (MSE)

##### We will start by building our network now, all the methods are explained in the docstrings, and the rest are explained below.

In [ ]:
import random
import numpy as np

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
            
        return a
    

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""

        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
                
            if test_data:
                print("Epoch {} : {} / {}".format(j,self.evaluate(test_data),n_test));
            else:
                print("Epoch {} complete".format(j))
                

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            # Adds the change in weight and biase required for each training example in mini-batch into nabla_b and nabla_w
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        
        # 3) Gradient Descent/ updating the weights and biases
        # v′ = v − η/m∇C
        self.weights = [w- (eta/len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b- (eta/len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)]
        

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # 1) Input
        activation = x
        activations = [x]          # List to store all the activations, layer by layer
        zs = []                    # List to store all the z vectors, layer by layer
        
        # 2a) Feedforward
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            
        # 2b) Backward pass/ Output Error
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        # 2c) Backpropogate the error for each layer 
        for l in range(2, self.num_layers):
            # Calculate error of second to last layer, than third to last layer and so on for all layers, backwards
            z = zs[-l]                               
            sp = sigmoid_prime(z)        # Derivative of simoid activation function
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp    # δ(l) =((w(l+1)^T * δ(l+1)) ⊙ σ′(z(l))
            nabla_b[-l] = delta                                           # ∂C/∂b(l) = δ
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())    # ∂C/∂w(l) = a(l−1) * δ(l)
            
        return (nabla_b, nabla_w)                       # Returns change in weights and biases for single training example
    

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        
        return (output_activations-y)

# Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

- The equation applied by the feed-forward method is: a′= σ(wa + b)
       where w is the weight matrix 
             b is the bias vector
          
- SGD method: The training_data is a list of tuples (x, y) representing the training inputs and corresponding desired outputs
   If the optional argument test_data is supplied, then the program will evaluate the network after each epoch of training, and    print out partial progress.
   
  In each epoch, it starts by randomly shuffling the training data, and then partitions it into mini-batches of the appropriate size. Then for each mini_batch we apply a single step of gradient descent. This is done by the code self.update_mini_batch(mini_batch, eta), which updates the network weights and biases according to a single iteration of gradient descent, using just the training data in mini_batch.
  
  
- update_mini_batch method: The most important line is delta_nabla_b, delta_nabla_w = self.backprop(x, y) which invokes the backpropagation algorithim for computing the gradients of the cost function. backprop method returns the appropriate gradient for the cost associated to the training example x.


- backprop method: Described below.

### Backpropogation Algorithim

Backpropagation is about understanding how changing the weights and biases in a network changes the cost function. Ultimately, this means computing the partial derivatives ∂C/∂w and ∂C/∂b. 
But to compute those, we first introduce an intermediate quantity, δ, which we call the error and we will compute it as avector for each layer.

The error, δ, is simply ∂C/∂a^L * σ′(z^L) (where we are multiplying how fast the cost is changing w.r.t last layer activation and how fast theactivation is changing.) 

Notation: z = wa+b

           where w is the weight matrix 
           b is the bias vector
           
a(L) means the activation of the last layer.
δ(l) means the error of the layer number l.

#### The equations for backpropagation

1) An equation for the error in the output layer, δL:
   δL = ∂C / ∂aL * σ′(z(L))
   
   or 
   
   δL = ∇aC ⊙ σ′(z(L))       
               
               ∇aC: is defined to be a vector whose components are the partial derivatives ∂C/∂aL
               You can think of ∇aC as expressing the rate of change of C with respect to the output activations.
               
               ∇aC = (a(L)−y) as C = 1/2 * ∥y − a(L)∥^2
               
               ⊙ - is the Hadamard product, i.e. the component by component product of the vectors
   
2) An equation for the error δ(l) in terms of the error in the next layer, δ(l+1):
    δ(l) = ((wl+1)^T * δ(l+1)) ⊙ σ′(z(l))
    
                
        When we apply the transpose weight matrix, (wl+1)T, we can think intuitively of this as moving the error backward      
        through the network, giving us some sort of measure of the error at the output of the lth layer. We then take the 
        Hadamard product ⊙σ′(zl). This moves the error backward through the activation function in layer l, giving us the error
        δ(l) in the weighted input to layer l.
        
        
3) An equation for the rate of change of the cost with respect to any bias in the network:
    ∂C/∂b = δ
    
4) An equation for the rate of change of the cost with respect to any weight in the network:
    ∂C/∂w(l) = a(l−1) * δ(l)
    
- Note: Equations 3 and 4 are the main equations and it these that go into the ∇C gradient vector during gradient descent
    v′ = v − η∇C
    Equations 1 and 2 are just intermidiates and that help us to compute equations 3 and 4.


###### All these equations are obtained by chain rule.

#### The algorithim: 

1) Input x: Set the corresponding activation a1 for the input layer.

2) Feedforward: For each l=2,3,…,L compute z = wa + b and a = σ(z).

3) Output error δ(L): Compute the vector δ(L) = ∇aC ⊙ σ′(z(L)); this is the output vector of the last layer.

4) Backpropagate the error: For each l=L−1,L−2,…,2 compute δ(l) =((w(l+1)T * δ(l+1)) ⊙ σ′(z(l)).

5) Output: The gradient of the cost function is given by ∂C/∂w(l) = a(l−1) * δ(l) and ∂C/∂b(l) = δ(l).

We compute the error vectors δ(l) backward, starting from the final layer. To understand how the cost varies with earlier weights and biases we need to repeatedly apply the chain rule, working backward through the layers to obtain usable expressions.




###### The backpropagation algorithm computes the gradient of the cost function for a single training example.

We combine backpropagation with stochastic gradient descent, in which we compute the gradient for many training examples. In particular, given a mini-batch of m training examples, the following algorithm applies a gradient descent learning step based on that mini-batch:

1) Input a set of training examples

2) For each training example x: Set the corresponding input activation a(x,1), and perform the following steps:

        2a) Feedforward: For each l=2,3,…,L compute z = wa + b and a = σ(z).
        
        2b) Output error δ(x,L): Compute the vector δ(x,L) = ∇aCx ⊙ σ′(z(x,L))
        
        2c) Backpropagate the error: For each l=L−1,L−2,…,2 compute δ(x,l) = ((wl+1)T * δ(x,l+1)) ⊙ σ′(z(x,l))
        
3) Gradient descent: For each l=L,L−1,…,2 update the weights and biases using ∂C/∂w and ∂C/∂b

### Phew! Well, now let us take a look at how our model performs.

Let us start by loading in the MNIST data:

In [2]:
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

### Network A with 30 hidden neurons (95% accuracy)

###### We will set up a Network with 30 neurons in the single hidden layer:

In [3]:
net = Network([784, 30, 10])

Finally, we'll use stochastic gradient descent to learn from the MNIST training_data over 30 epochs, with a mini-batch size of 10, and a learning rate of η=3.0

In [4]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0 : 9079 / 10000
Epoch 1 : 9217 / 10000
Epoch 2 : 9308 / 10000
Epoch 3 : 9254 / 10000
Epoch 4 : 9357 / 10000
Epoch 5 : 9356 / 10000
Epoch 6 : 9401 / 10000
Epoch 7 : 9417 / 10000
Epoch 8 : 9384 / 10000
Epoch 9 : 9428 / 10000
Epoch 10 : 9455 / 10000
Epoch 11 : 9442 / 10000
Epoch 12 : 9455 / 10000
Epoch 13 : 9449 / 10000
Epoch 14 : 9461 / 10000
Epoch 15 : 9473 / 10000
Epoch 16 : 9461 / 10000
Epoch 17 : 9454 / 10000
Epoch 18 : 9447 / 10000
Epoch 19 : 9475 / 10000
Epoch 20 : 9471 / 10000
Epoch 21 : 9485 / 10000
Epoch 22 : 9475 / 10000
Epoch 23 : 9493 / 10000
Epoch 24 : 9476 / 10000
Epoch 25 : 9463 / 10000
Epoch 26 : 9462 / 10000
Epoch 27 : 9489 / 10000
Epoch 28 : 9471 / 10000
Epoch 29 : 9482 / 10000


###### The accuracy we get with a single hidden layer with 10 hidden neurons is 94.8%

### Network B with 100 hidden neurons (97% accuracy)

###### Changing the number of hidden neurons to 100 increases the complexity of the the neural net and thus we are expecting a bump to our 95% accuracy.

In [5]:
net2 = Network([784, 100, 10])

In [6]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
net2.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0 : 5667 / 10000
Epoch 1 : 6629 / 10000
Epoch 2 : 6827 / 10000
Epoch 3 : 7590 / 10000
Epoch 4 : 7617 / 10000
Epoch 5 : 7688 / 10000
Epoch 6 : 8624 / 10000
Epoch 7 : 8687 / 10000
Epoch 8 : 9483 / 10000
Epoch 9 : 9554 / 10000
Epoch 10 : 9533 / 10000
Epoch 11 : 9561 / 10000
Epoch 12 : 9579 / 10000
Epoch 13 : 9591 / 10000
Epoch 14 : 9614 / 10000
Epoch 15 : 9618 / 10000
Epoch 16 : 9619 / 10000
Epoch 17 : 9627 / 10000
Epoch 18 : 9628 / 10000
Epoch 19 : 9616 / 10000
Epoch 20 : 9627 / 10000
Epoch 21 : 9657 / 10000
Epoch 22 : 9636 / 10000
Epoch 23 : 9649 / 10000
Epoch 24 : 9651 / 10000
Epoch 25 : 9660 / 10000
Epoch 26 : 9642 / 10000
Epoch 27 : 9660 / 10000
Epoch 28 : 9660 / 10000
Epoch 29 : 9653 / 10000


###### As expected, increasing the number of neurons results an improvement in the perfomance of our network as the accuracy improves from 94.8% to 96.5%